# Li+ Analysis Using pdbionsurvey

`pdbionsurvey` is a library that seeks to quantify the interactions between cations and polarized atoms in proteins, namely oxygen. The library attempts to simplify the amount of ad hoc coding that must be done while still maintaining flexibility in its use. Here, we will demonstrate the use of `pdbionsurvey` in analyzing cation/oxygen interactions in proteins. We will use lithium ions (Li+) as our example ion since it is less common than Na+ or K+ which decreases the amount of data to process and therefore the computation time.

### First Things First: imports and Variables

##### Necessary import statements
We need to import a few packages to make things more readily useable.

In [1]:
# import the library that is being demonstrated
import pdbionsurvey.coordination
import pdbionsurvey.collection
import pdbionsurvey.analysis

# needed to make protein universes
import MDAnalysis as mda

# needed to make and use sims
import mdsynthesis as mds

# needed for path definitions
import os

# needed for displaying plots
import matplotlib.pyplot as plt
%matplotlib inline

# needed for simplicity
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python2.7/dist-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


##### Path definitions

We should put all of our files in directories and sort them nicely. In general, these directories can live anywhere, which allows people to use the package to suit their own needs.

In [2]:
# directory for the pdbs
path = mds.Tree('/nfs/homes/kreidy/Projects/PDB_Ion_Survey/pdbs/')

# directory for the sims
# it is important that this directory is empty,
# otherwise, mislabeling may happen later
simdir = path['sims_folder/']

# directory for DataFrames
datapath = path['dataframes']

In [3]:
simdir.trees[:3].draw()

CL/
 +-- 4QH5/
 |   +-- 4QH5.pdb
 |   +-- Sim.79a40f01-dcd0-46b1-b382-379767c5d244.json
 +-- 4H7J/
 |   +-- Sim.3bbf3409-9501-4f13-b3f8-524ad5e41a65.json
 |   +-- 4H7J.pdb
 +-- 5LDM/
 |   +-- Sim.afdaa587-de3f-4532-9147-f5995df744fe.json
 |   +-- 5LDM.pdb
 +-- 5CAI/
 |   +-- 5CAI.pdb
 |   +-- Sim.8dcec6cd-c6d1-41cb-ad65-d60b7982b993.json
 +-- 1C63/
 |   +-- Sim.5b887b23-59bf-47da-9def-aa5e9fe0d2a0.json
 |   +-- 1C63.pdb
 +-- 3MOR/
 |   +-- 3MOR.pdb
 |   +-- Sim.c03ea5a8-8988-4b4b-b619-8b0600db42a7.json
 +-- 4FU3/
 |   +-- Sim.cc83d6ba-ea98-4b2e-b1ad-8d589aead616.json
 |   +-- 4FU3.pdb
 +-- 4DFR/
 |   +-- 4DFR.pdb
 |   +-- Sim.4d675fc9-91a0-4a7f-8d32-bae12815185b.json
 +-- 1PS9/
 |   +-- Sim.4823a2ca-a57b-439d-ad93-28505bec5d2a.json
 |   +-- 1PS9.pdb
 +-- 5LA8/
 |   +-- 5LA8.pdb
 |   +-- Sim.c5b56828-a57e-4dea-87af-55bf3c95dbc1.json
 +-- 3IIZ/
 |   +-- 3IIZ.pdb
 |   +-- Sim.bab4957f-830a-4f5d-a3d7-367e03fd5e9e.json
 +-- 5EVL/
 |   +-- 5EVL.pdb
 |   +-- Sim.b639e8dc-ec39-4437-beb0-6b8ae2

### Obtaining Data from the PDB

### Get PDB ids
We need to query the PDB for all of the PDB ids cooresponding to files that contain Li+ ions.

`pdbionsurvey.collection.get_pdb_ids` has other parameters that are left with their defaults:  
`containsProtein=True` will filter out any molecules that do not contain proteins,  
`containsDNA=False` will filter out any molecules that contain DNA (because it's set to false),  
`containsRna=False` will filter out any molecules that contain RNA,  
and `containsHybrid=False` will filter out any molecules that contain multiple molecule types.

In [4]:
# find and store the PDB ids
ids = pdbionsurvey.collection.get_pdb_ids('Li')

# display the number of PDB ids
print len(ids)

63


### Download .pdb files corresponding to PDB ids

Good to break up scripts into modules and functions to demonstrate how components work to produce data. Separating querying from analysis allows for more understanding and flexibility in use.

In [5]:
# download each .pdb file individually
for pdbid in ids:
    pdbionsurvey.collection.get_pdb_file(pdbid, path.abspath)

### Making Sims

We make sims using `mdsynthesis` to store our data so that we can easily access it in the future. We can use `pdbionsurvey.analysis.make_sims` to quickly make them consistently for the purpose of coorcination analysis.

In [10]:
pdbs = path.glob('*.pdb')
pdbs = pdbs[[pdb.name in [r + '.pdb' for r in ids] for pdb in pdbs]]

In [11]:
# make sims
sims = pdbionsurvey.analysis.make_sims(simdir, pdbs)

In [9]:
simdir.trees[:3].draw()

KeyboardInterrupt: 

### Making and Storing Radial Distribution Functions

We use MDAnalysis to create a protein Universe and to select the ions that we care about. From there, we can use `pdbionsurvey.coordination.en` to create a radial distribution function for each .pdb file. We then store our data as .csv files for easy access and sorting.

In [12]:
pdbionsurvey.analysis.define_universe(sims, pdbs)

In [ ]:
simdir.trees[:3].draw()

In [13]:
for i in sims:
    # make a protein Universe
    u = mda.Universe(i.universedef.topology)

    # select Li atoms
    ions = u.select_atoms('name LI')

    for ion in ions:
        # make radial distribution function
        df = pdbionsurvey.coordination.en(u, ion)
        
        # make directory if it doesn't exist
        i['coordination/LI/'].make()

        # store radial distribution function in .csv file
        df.to_csv(i['coordination/LI/{}.csv'.format(ion.number)].abspath)

In [ ]:
simdir.trees[:3].draw()

### Labeling Sims

We should tag all of the sims with the ions they contain, as well as add categories for resolution information and for whether or not water is explicitly contained in the file.

In [41]:
b = mds.discover(simdir)

In [43]:
len(b[b.tags['Li']])

63

In [37]:
# label sims
pdbionsurvey.analysis.sim_labeling(sims, ionname='Li')

### Collecting Sims

Using mdsynthesis, we can create a bundle of all of the sims in simdir. We already have this bundle in `sims`, but we might want to do this if we have other sims in our sim directory, or if we want to grab them later from another notebook.

We can then sort these sims using their new tags and categories.

In [ ]:
# create a Bundle of sims in simdir
b = mds.discover(simdir)

### Radial Distribution: g(r)

A g(r) is a function relating the distance away from something to the density of something around it. In this case, a g(r) is the relationship between the distance away from a Li+ ion and the density of oxygen atoms. Each point on the graph that we are making can be visualized as a sphere around the ion with the density of oxygens corresponding to the height of the point.

In [ ]:
m, denses = pdbionsurvey.coordination.gee(b, 'LI')

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Distance ($\AA$)')
ax.set_ylabel('Oxygen Density')
ax.set_xlim((1,6))
ax.plot(m, denses)

### Finding Peaks

It may also be useful to find where these solvation shells are exactly. By finding the peaks and troughs of a radial density distribution function, we can see in what bands most of the oxygen atoms lie. The troughs, where relatively few oxygen atoms are, can therefore be used as one boundary of a solvation shell.

In [ ]:
# get peak information
m, dense, peaks, mins = pdbionsurvey.analysis.get_peaks(b, 'Li', mindist=.2)

In [ ]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(1,1,1)
ax.set_xlim((1, 6))
ax.set_xlabel('Distance ($\AA$)')
ax.set_ylabel('Oxygen Density')
ax.plot(m, dense)
ax.plot(m[peaks], dense[peaks], 'o')
ax.plot(m[mins], dense[mins], 'o')

### Closest Oxygen Distances

It is useful in some applications to know the average distance of the n$^\text{th}$ closest oxygen atom to the ion, particularly for those n's that correspond to the oxygen atoms in the first solvation shell.

Here we'll look at the closest six oxygen atoms to all of the Li+ ions in our sample.

In [ ]:
# get closest oxygen information
oxys = pdbionsurvey.analysis.closest_oxy_distance(b, ['Li'], [2.5])

In [ ]:
oxys[0].head()

In [ ]:
len(oxys[0])

In [ ]:
ax = pdbionsurvey.analysis.graph_closest_oxy_distances(oxys[0], axlim=(1, 6))

# Conclusion

We can see here that some of the plots that we generated are rather choppy. This is most likely because of the relative lack of data for Li+. It is important to note that the emphasis here should not be on the science but on the computation and using the library, so we won't discuss the analysis of our results with too much depth. It is important to know that the library is structured in such a way that it has a lot of dependency on packages like `MDAnalysis`, `mdsynthesis`, and `peakutils`, and it makes a lot of assumptions about the structure of the Sims it uses, but once these other packages are installed, `pdbionsurvey`  makes it easy to make Sims in the way that it wants.